In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
models = pd.read_excel('YOUR PATH TO PRS APPLIED PAPERS',sheet_name='Method papers')
models = models[:-4]
models = pd.DataFrame(models)

In [ ]:
# helper will be used when constructing AA0 and WW0 in order to ensure the data get in 
# correct columns. allRankings will be use to facilitate the violinplot calculation
helper = {}
allRankings = {}
for x in range(0,models.shape[0]):
    allRankings[models[models.columns[0]].iloc[x]] = []
    helper[models[models.columns[0]][x]] = x

In [ ]:
AA0 = pd.DataFrame(columns=np.asarray(models['Method']))
WW0 = pd.DataFrame(columns=np.asarray(models['Method']))

In [ ]:
#This function creates the AA0 and WW0 matrix.
def getInfo(df,name):
    global AA0
    global WW0
    r = df.shape[0]
    c = df.shape[1]
    #Loop over each row
    #In each row loop method
    #Compare each method with the methods not yet compared
    #If the comparison is not 0, add that comparison to AA0 and WW0
    #At the end of the loop, add the same data to the phenotype level AA0 and WW0 matrix
    for x in range(0,r):
        for y in range(1,c):
            comparer = df.columns[y]
            for z in range(y+1,c):
                comparie = df.columns[z]
                if (not ( pd.isna(df[comparer].iloc[x]) or pd.isna(df[comparie].iloc[x]) )):
                    comparerVal = df[comparer].iloc[x]
                    comparieVal = df[comparie].iloc[x]
                    
                    diff = comparerVal-comparieVal

                    if diff != 0:
                        #AAO matrix calculation
                        tmp1 = helper[comparer]
                        tmp2 = helper[comparie]
                        arr = np.zeros(14)
                        arr[tmp1] = 1
                        arr[tmp2] = 1
                        df2 = pd.DataFrame([arr],columns=np.asarray(models['Method']))
                        AA0 = pd.concat([AA0,df2],ignore_index=True)

                        #WW0 matrix calculation
                        arr2 = np.zeros(14)
                        arr2[tmp1 if diff > 0 else tmp2] = 1
                        df3 = pd.DataFrame([arr2],columns=np.asarray(models['Method']))
                        WW0 = pd.concat([WW0,df3],ignore_index=True)
                        
                        #Phenotype level AA0 and WW0 information
                        phen = df[df.columns[0]].iloc[x]
                        if traitMap.keys().__contains__(phen):
                            traitsDiseaseCount[traitMap[phen]].add(name)
                            traits[traitMap[phen]][0] = pd.concat([traits[traitMap[phen]][0],df2],ignore_index=True)
                            traits[traitMap[phen]][1] = pd.concat([traits[traitMap[phen]][1],df3],ignore_index=True)

In [ ]:
#We had datasets of one phenotype which were compared on a cohort level
#Becuase the cohorts aren't phenotypes, the data from these data sets weren't added
#to the phenotype level AA0 and WW0 matrix (but they were already added to the overall
#AA0 and WW0 matrix). This function adds the cohort level analysis to the phenotype
#specific AA0 and WW0
def getInfoCohort(df,dfOfCohort):
    global AA0
    global WW0

    r = df.shape[0]
    c = df.shape[1]

    for x in range(0,r):
        for y in range(1,c):
            comparer = df.columns[y]
            for z in range(y+1,c):
                comparie = df.columns[z]
                if (not ( pd.isna(df[comparer].iloc[x]) or pd.isna(df[comparie].iloc[x]) )):
                    comparerVal = df[comparer].iloc[x]
                    comparieVal = df[comparie].iloc[x]
                    
                    diff = comparerVal-comparieVal

                    if not (diff == 0):
                        #Adding in the AAO matrix calculation\
                        tmp1 = helper[comparer]
                        tmp2 = helper[comparie]
                        arr = np.zeros(14)
                        arr[tmp1] = 1
                        arr[tmp2] = 1
                        df2 = pd.DataFrame([arr],columns=np.asarray(models['Method']))
                        dfOfCohort[0] = pd.concat([dfOfCohort[0],df2],ignore_index=True)

                        ##Adding the WW0 matrix calculation
                        arr2 = np.zeros(14)
                        arr2[tmp1 if diff > 0 else tmp2] = 1
                        df3 = pd.DataFrame([arr2],columns=np.asarray(models['Method']))
                        dfOfCohort[1] = pd.concat([dfOfCohort[1],df3],ignore_index=True)
    return dfOfCohort

In [ ]:
dfM = pd.read_excel('YOUR PATH TO PRS APPLIED PAPERS',sheet_name=None)
dfM.pop('Method papers')
dfM.pop('Application and benchmarking')

In [ ]:
#each paper had different notation for phenotypes. This normalizes it.
traitMap = {'AD': "Alzheimer's Disease",
 'ADHD': "Attention-deficit/hyperactivity disorder",
 'ADNI123-M1': "Alzheimer's Disease",
 'ADNI123-M2': "Alzheimer's Disease",
 'ADNI123-M3': "Alzheimer's Disease",
 'ADNI123-M4': "Alzheimer's Disease",
 'ADNI123-M5': "Alzheimer's Disease",
 'AFRBMI': "Body Mass Index",
 'AFRHDL': "High-Density Lipoprotein",
 'AFRHeight': "Height",
 'AFRLDL': "Low-Density Lipoprotein",
 'AFRTC': "Cholesterol",
 'AFRlogTG': "Triglyceride",
 'AM': "Age at Menarche",
 'AN': "Angina",
 'AS': "Asthma Disease",
 'ATH': "Asthma Disease",
 'African AmericanAny CVD': "Cardio Vascular Disease",
 'African AmericanDepression': "Depression",
 'African AmericanHeart metabolic disease burden': "Heart Metabolic Disease Burden",
 'African AmericanHeight': "Height",
 'African AmericanMigraine Diagnosis':"Migraine Diagnosis",
 'African AmericanMorning Person':"Morning Person",
 'African AmericanSBMN':"Sing Back Musical Note",
 'Age-related macular degeneration': "Age Related Macular Degeneration",
 'Alzheimer’s disease': "Alzheimer's Disease",
 'Asthma': "Asthma Disease",
 'Asthma-BBJ': "Asthma Disease",
 'Asthma-Consortium':"Asthma Disease",
 'Asthma-FinnGen':"Asthma Disease",
 'Asthma-UKB':"Asthma Disease",
 'Asthma-UKBB':"Asthma Disease",
 'Atrial fibrillation': "Atrial Fibrillation",
 'BFP': "Body Fat Percentage",
 'BIP': "Bipolar Disorder",
 'BMD': "Bone Mineral Density",
 'BMI': "Body Mass Index",
 'BMR': "Basal Metabolic Rate",
 'BRCA': "Breast Cancer",
 'BW': "Birth Weight",
 'BioFINDER-M1':"Alzheimer's Disease",
 'BioFINDER-M2':"Alzheimer's Disease",
 'BioFINDER-M3':"Alzheimer's Disease",
 'BioFINDER-M4':"Alzheimer's Disease",
 'BioFINDER-M5':"Alzheimer's Disease",
 'Bipolar disorder': "Bipolar Disorder",
 'Bladder Cancer': "Bladder Cancer",
 'Bowel cancer': "Bowel Cancer",
 'BrCa': "Breast Cancer",
 'BrCa-BBJ': "Breast Cancer",
 'BrCa-Consortium': "Breast Cancer",
 'BrCa-FinnGen': "Breast Cancer",
 'BrCa-UKB': "Breast Cancer",
 'BrCa-UKBB': "Breast Cancer",
 'Breast Cancer': "Breast Cancer",
 'Breast cancer': "Breast Cancer",
 'CAD': "Coronary Artery Disease",
 'CAD-BBJ': "Coronary Artery Disease",
 'CAD-Consortium': "Coronary Artery Disease",
 'CAD-FinnGen': "Coronary Artery Disease",
 'CAD-UKB': "Coronary Artery Disease",
 'CAD-UKBB': "Coronary Artery Disease",
 'CKD': "Chronic Kidney Disease",
 'Cardiovascular disease': "Cardio Vascular Disease",
 'Cataract-BBJ':"Cataracts",
 'Cataract-UKB':"Cataracts",
 'Celiac Disease':"Celiac Diesease",
 'Celiac disease':"Celiac Diesease",
 'Colorectal Cancer':"Colorectal Cancer",
 'Coronary artery disease': "Coronary Artery Disease",
 'Crohns Disease':"Crohn's Disease",
 'Crohn’s disease':"Crohn's Disease",
 'DBP': "Diastolic Blood Pressure",
 'DBT': "Diabetes",
 'DEP': "Depression",
 'DFI': "Dried Fruit Intake",
 'Depression': "Depression",
 'Distolic Blood Pressure': "Diastolic Blood Pressure",
 'EASHDL': "High-Density Lipoprotein",
 'EASLDL': "Low-Density Lipoprotein",
 'EASTC': "Cholesterol",
 'EASlogTG': "Triglyceride",
 'EC': "Eosinophil Count",
 'EOS':"Eosinophil Count",
 'ES': "Ever Smoked",
 'East AsianAny CVD': "Cardio Vascular Disease",
 'East AsianDepression': "Depression",
 'East AsianHeart metabolic disease burden': "Heart Metabolic Disease Burden",
 'East AsianHeight': "Height",
 'East AsianMigraine Diagnosis': "Migraine Diagnosis",
 'East AsianMorning Person': "Morning Person",
 'East AsianSBMN': "Sing Back Musical Note",
 'Epithelial ovarian cancer': "Epithelia Ovarian Cancer",
 'FEV': "Forced Expiratory Volume",
 'FFI': "Fresh Fruit Intake",
 'FFR': "(Forced Expiratory Volume in 1 second)/(Forced Vital Capacity)",
 'FVC': "Forced Vital Capacity",
 'GCSE': "General Certificate of Secondary Education",
 'GO': "Gout",
 'Gastric Cancer-BBJ': "Gastric Cancer",
 'Gastric Cancer-UKB': "Gastric Cancer",
 'Glaucoma-BBJ': "Glaucoma",
 'Glaucoma-UKB': "Glaucoma",
 'Gout': "Gout",
 'HA': "Headache",
 'HBP': "High Blood Pressure",
 'HC': "Hip Circumference",
 'HDL': "High-Density Lipoprotein",
 'HDL Cholesterol': "High-Density Lipoprotein",
 'HDL-UKBB': "High-Density Lipoprotein",
 'HDL-constoritum': "High-Density Lipoprotein",
 'HP': "Haematocrit Percentage",
 'HT': "Height",
 'HbA1c': "Hemoglobin A1c",
 'Height': "Height",
 'Height-UKBB': "Height",
 'Height-consortium': "Height",
 'Hypercholesterolemia': "Hypercholesterolemia",
 'Hypertension': "Hypertension",
 'Hyperthyroidism-BBJ':"Hyperthyroidism",
 'Hypertriglyceridemia': "Hypertriglyceridemia",
 'Hypothyroidism-BBJ':"Hypothyroidism",
 'Hypothyroidism-UKB':"Hypothyroidism",
 'Hyptertension': "Hypertension",
 'IBD': "Inflammatory Bowel Disease",
 'Intelligence': "Intelligence",
 'Ischemic stroke': "Ischemic Stroke",
 'Kidney Cancer': "Kidney Cancer",
 'LC': "Lymphocyte Count",
 'LDL': "Low-Density Lipoprotein",
 'LDL Cholesterol': "Low-Density Lipoprotein",
 'LDL-C': "Low-Density Lipoprotein",
 'LDL-UKBB': "Low-Density Lipoprotein",
 'LDL-consortium': "Low-Density Lipoprotein",
 'LatinoAny CVD': "Cardio Vascular Disease",
 'LatinoDepression': "Depression",
 'LatinoHeart metabolic disease burden': "Heart Metabolic Disease Burden",
 'LatinoHeight': "Height",
 'LatinoMigraine Diagnosis': "Migraine Diagnosis",
 'LatinoMorning Person': "Morning Person",
 'LatinoSBMN': "Sing Back Musical Note",
 'Lung Cancer': "Lung Cancer",
 'MBI-UKBB': "Body Mass Index",
 'MBI-consortium': "Body Mass Index",
 'MC': "Monocyte Count",
 'MCH': "Mean Corpuscular Hemoglobin",
 'MCHC': "Mean Corpuscular Hemoglobin Concentration",
 'MCV': "Mean Corpuscular Volume",
 'MDD': "Depression",
 'MP': "Morning Person",
 'MY': "Myxedema",
 'Melanoma': "Melanoma",
 'MultiScler': "Multiple Sclerosis",
 'NC': "Neutrophil Count",
 'OA': "Osteoarthritis",
 'Obesity': "Obesity",
 'Osteoporosis': "Osteoporosis",
 'Osteoporosis-BBJ': "Osteoporosis",
 'Osteoporosis-UKB': "Osteoporosis",
 'Ovary Cancer': "Ovarian Cancer",
 'PC': "Platelet Count",
 'PLT': "Platelet Count",
 'PRCA': "Prostate Cancer",
 'Pancreas Cancer': "Pancreatic Cancer",
 'Parkinson’s disease': "Parkinson's Disease",
 'PrCa': "Prostate Cancer",
 'PrCa-Consortium': "Prostate Cancer",
 'PrCa-FinnGen': "Prostate Cancer",
 'PrCa-UKBB': "Prostate Cancer",
 'Primary open angle glaucoma': "Primary Open Angle Glaucoma",
 'Prostate Cancer': "Prostate Cancer",
 'Prostate cancer':"Prostate Cancer",
 'Psoriasis': "Psoriasis",
 'QU': "Qualification",
 'RA': "Rheumatoid Arthirits",
 'RBC': "Red Blood Cell Count",
 'RBCC':"Red Blood Cell Count",
 'RDW': "Red Blood Cell Distribution Width",
 'RheuArth': "Rheumatoid Arthirits",
 'Rheumatoid arthritis': "Rheumatoid Arthirits",
 'SAF': "Salt Added to Food",
 'SASHDL': "High-Density Lipoprotein",
 'SASLDL': "Low-Density Lipoprotein",
 'SASTC': "Cholesterol",
 'SASlogTG': "Triglyceride",
 'SBP': "Systolic Blood Pressure",
 'SCZ': "Schizophrenia",
 'SCZ-MGS': "Schizophrenia",
 'SCZ-ISC': "Schizophrenia",
 'CHOL': "Cholesterol",
 'TRIG': "Triglyceride",
 'SH': "Height",
 'SN': "Snoring",
 'SS': "Smoking Status",
 'SU': "Sodium in Urine",
 'Schizophrenia': "Schizophrenia",
 'Sleep Duration': "Sleep Duration",
 'South AsianAny CVD': "Cardio Vascular Disease",
 'South AsianDepression': "Depression",
 'South AsianHeart metabolic disease burden': "Heart Metabolic Disease Burden",
 'South AsianHeight': "Height",
 'South AsianMigraine Diagnosis': "Migraine Diagnosis",
 'South AsianMorning Person': "Morning Person",
 'South AsianSBMN': "Sing Back Musical Note",
 'Stroke': "Stroke",
 'Systemic lupus erythematosus': "Systemic Lupus Erythematosus",
 'Systollic Blood Pressure': "Systolic Blood Pressure",
 'T1B': "Type One Balding",
 'T1D': "Type One Diabetes",
 'T2D': "Type Two Diabetes",
 'T2D-BBJ': "Type Two Diabetes",
 'T2D-Consortium': "Type Two Diabetes",
 'T2D-FinnGen': "Type Two Diabetes",
 'T2D-UKB': "Type Two Diabetes",
 'T2D-UKBB': "Type Two Diabetes",
 'T2d': "Type Two Diabetes",
 'TA': "Tanning Ability",
 'TC': "Cholesterol",
 'TC-UKBB': "Cholesterol",
 'TC-consortium': "Cholesterol",
 'TE': "Tense",
 'TFP': "Trunk Fat Percentage",
 'TG': "Triglyceride",
 'Total Cholesterol': "Cholesterol",
 'Total cholesterol': "Cholesterol",
 'Triglyceride': "Triglyceride",
 'Triglycerides': "Triglyceride",
 'Ulcerative colitis': "Ulcerative Colitis",
 'Urate':  "Urate",
 'VMS': "Supplementary Vitamin and Mineral",
 'Venous thromboembolic disease': "Venous Thromboembolic Disease",
 'WBC': "White Blood Cell Count",
 'WBCC': "White Blood Cell Count",
 'WC': "Waist Circumference",
 'WHR': "Waist Hip Ratio",
 'eGFR': "Estimated Glomerular Filtration Rate",
 'logTG-UKBB': "Triglyceride",
 'logTG-consortium': "Triglyceride",
 'Height-H&R Study': "Height",
 'BMI-H&R Study': "Body Mass Index",
 'Height-Estonian BioBank': "Height",
 'BMI-H&R Estonian Biobank': "Height",
 'T2D AUC': "Type Two Diabetes",
 }

In [ ]:
#Creates the phenotype level AA0 and WW0 matrices
traits = {}
traitsDiseaseCount = {}
for x in set(traitMap.values()):
    traits[x] = [pd.DataFrame(columns=np.asarray(models['Method']) ),pd.DataFrame(columns=np.asarray(models['Method']) )]
    traitsDiseaseCount[x] = set()

In [ ]:
def colSums(AA0):
    answer = []
    for x in AA0.columns:
        answer.append(sum(AA0[x]))
    return answer

In [ ]:
#We remove methods which weren't compared enough, but we can't simply drop the column as 
#the row of that methods comparisons still exist
#To fix this, we have to find each row where this method was compared and remove it before
#we can drop the column
def cleanUp(AA, WW, col):
    remVec = []
    for x in range(len(AA[col])):
        if AA[col].iloc[x] == 1:
            remVec.append(x)
    AA = AA.drop(remVec)
    WW = WW.drop(remVec)

    AA.index = range(len(AA[AA.columns[0]]))
    WW.index = range(len(AA[AA.columns[0]]))

In [ ]:
#Implementation of the algorithm described in the spectral method ranking paper
def stationaryCalc(AA,WW):
    fAvec2 = AA.shape[0] + 2
    dval2 = max(colSums(AA))*2  
    P2 = np.zeros( (AA.shape[1], AA.shape[1]) )
    for i in range(AA.shape[1]): 
        for j in range(AA.shape[1]):
            if j != i:
                tmp1 = np.asarray(AA[AA.columns[i]]) 
                tmp2 = np.asarray(AA[AA.columns[j]])
                tmp3 = np.asarray(WW[WW.columns[j]]) 
                tmp4 = np.multiply(tmp1,tmp2)
                P2[i][j] = sum(np.multiply(tmp4,tmp3)) / (dval2*fAvec2)
        P2[i][i] = 1 - sum(P2[i])
    
    evals, evecs = np.linalg.eig(P2.T)
    evec1 = evecs[:,np.isclose(evals, 1)]
    evec1 = evec1[:,0]
    stationary = evec1 / evec1.sum()
    return stationary.real

In [ ]:
def rankingCalculation(AA, WW, trait):
    removes = []
    #This ensures we have removed methods which were only compared a few times
    doRemove = True
    while doRemove:
        for x in AA.columns:
            if sum(AA[x]) < 10:
                removes.append(x)
        if len(removes) == 0:
            doRemove = False
        for x in removes:
            cleanUp(AA,WW,x)
            AA=AA.drop(x,axis=1)
            WW=WW.drop(x,axis=1)
        removes = []
    #If we removed all the methods / dropped all the rows we can't do anything
    if AA.shape[0] == 0 or AA.shape[1] < 2:
        print('not enough data!', trait)
        return
    
    #Doing this filtering didn't change much
    # if len(traitsDiseaseCount[trait]) < 3:
    #     print('only two data sets', trait)
    #     return

    #Ranking has to be done on a loop because some methods never lost,
    #so we need rank that method as first and re-run the algorithm without that method
    #to get the second - last ranking    
    indexed = []
    doCalc = True
    while doCalc:
        doCalc = False
        if AA.shape[1] == 1:
            indexed.append((AA.columns[0],0))
            stationary = []
            break
        
        stationary = stationaryCalc(AA,WW)    
        for x in range(len(stationary)):
            if stationary[x] == 1:
                method = AA.columns[x]
                indexed.append((method,len(stationary)))
                cleanUp(AA,WW,method)
                AA=AA.drop(method,axis=1)
                WW=WW.drop(method,axis=1)
                doCalc = True

    for x in range(len(stationary)):
        indexed.append((AA.columns[x], stationary[x]))
    
    indexed.sort(key=lambda tup: -tup[1]) 

    for y in range(len(indexed)):
        allRankings[indexed[y][0]].append((y+1)/len(indexed))


In [ ]:
for x in dfM.keys():
    name = x.split("-")[len(x.split("-")) - 1]
    getInfo(dfM[x],name)

traits["Alzheimer's Disease"] = getInfoCohort(dfM['AUC-Alzheimer-YuexuanXu'],traits["Alzheimer's Disease"])
traits["Alzheimer's Disease"] = getInfoCohort(dfM['AUC-Alzheimer-GannaLeonenk'],traits["Alzheimer's Disease"])
traits["Alzheimer's Disease"] = getInfoCohort(dfM['AUC-alz-GWAS-EftychiaBellou'],traits["Alzheimer's Disease"])
traits["Alzheimer's Disease"] = getInfoCohort(dfM['AUC-alz-RD-GWAS-EftychiaBellou'],traits["Alzheimer's Disease"])
traitsDiseaseCount["Alzheimer's Disease"].add("YuexuanXu")
traitsDiseaseCount["Alzheimer's Disease"].add("GannaLeonenk")
traitsDiseaseCount["Alzheimer's Disease"].add("EftychiaBellou")

traits["Depression"] = getInfoCohort(dfM['AUC-MDD-ByCohort-GuiyanNi'],traits["Depression"])
traitsDiseaseCount["Depression"].add("GuiyanNi")

traits["Schizophrenia"] = getInfoCohort(dfM['AUC-SCZ-ByCohort-GuiyanNi'],traits["Schizophrenia"])
traitsDiseaseCount["Schizophrenia"].add("GuiyanNi")

In [ ]:
for trait in traits:
    rankingCalculation(traits[trait][0],traits[trait][1],trait)

In [ ]:
#Creates input for violin plot calc in R
violin = pd.DataFrame(columns=["Method","Ranking"])
for x in allRankings.keys():
    vec = allRankings[x]
    for y in vec:
        violin=pd.concat([violin,pd.DataFrame([[x,y]], columns=["Method","Ranking"])],ignore_index=True)
violin.to_csv("violin.csv")

In [ ]:
AA0.to_csv('aa0AppliedPapers.csv',index=False)
WW0.to_csv('ww0AppliedPapers.csv',index=False)

In [ ]:
phenoTYPE = {'Tense': 'Binary',
 'Fresh Fruit Intake': 'Binary',
 'Body Fat Percentage': 'Continous Measure',
 'Basal Metabolic Rate': 'Continous Measure',
 'Ovarian Cancer': 'Binary',
 'Attention-deficit/hyperactivity disorder': 'Binary',
 'Pancreatic Cancer': 'Binary',
 'Gout': 'Binary',
 'Waist Circumference': 'Continous Measure',
 'Hyperthyroidism': 'Binary',
 'Hypertriglyceridemia': 'Binary',
 'Body Mass Index': 'Continous Measure',
 'Bowel Cancer': 'Binary',
 'Glaucoma': 'Binary',
 'Multiple Sclerosis': 'Binary',
 "Crohn's Disease": 'Binary',
 'Stroke': 'Binary',
 'Bipolar Disorder': 'Binary',
 'Cardio Vascular Disease': 'Binary',
 'Systolic Blood Pressure': 'Continous Measure',
 'Height': 'Continous Measure',
 'Platelet Count': 'Discrete Count',
 'Type One Diabetes': 'Binary',
 'Neutrophil Count': 'Discrete Count',
 'Qualification': 'Binary',
 'Ever Smoked': 'Binary',
 'Salt Added to Food': 'Binary',
 'Ischemic Stroke': 'Binary',
 'Kidney Cancer': 'Binary',
 'Snoring': 'Binary',
 'Psoriasis': 'Binary',
 "Parkinson's Disease": 'Binary',
 'Red Blood Cell Count': 'Discrete Count',
 'Heart Metabolic Disease Burden': 'Continous Measure',
 'High Blood Pressure': 'Binary',
 'Type Two Diabetes': 'Binary',
 'Sodium in Urine': 'Continous Measure',
 'Colorectal Cancer': 'Binary',
 'Cataracts': 'Binary',
 'Haematocrit Percentage': 'Continous Measure',
 'Inflammatory Bowel Disease': 'Binary',
 'Monocyte Count': 'Discrete Count',
 'Waist Hip Ratio': 'Continous Measure',
 'Lymphocyte Count': 'Discrete Count',
 'Osteoarthritis': 'Binary',
 'Mean Corpuscular Volume': 'Continous Measure',
 'Forced Vital Capacity': 'Continous Measure',
 'Migraine Diagnosis': 'Binary',
 'White Blood Cell Count': 'Discrete Count',
 'Headache': 'Binary',
 'Diabetes': 'Binary',
 'Bone Mineral Density': 'Continous Measure',
 'Hypertension': 'Binary',
 'Triglyceride': 'Continous Measure',
 'Melanoma': 'Binary',
 'Gastric Cancer': 'Binary',
 'Ulcerative Colitis': 'Binary',
 'General Certificate of Secondary Education': 'Binary',
 'Celiac Diesease': 'Binary',
 'Birth Weight': 'Continous Measure',
 'Age Related Macular Degeneration': 'Binary',
 'Urate': 'Continous Measure',
 'Breast Cancer': 'Binary',
 'Trunk Fat Percentage': 'Continous Measure',
 'Eosinophil Count': 'Discrete Count',
 'Diastolic Blood Pressure': 'Continous Measure',
 'Lung Cancer': 'Binary',
 'Chronic Kidney Disease': 'Binary',
 'Coronary Artery Disease': 'Binary',
 'Hypercholesterolemia': 'Binary',
 '(Forced Expiratory Volume in 1 second)/(Forced Vital Capacity)': 'Continous Measure',
 'Schizophrenia': 'Binary',
 'Type One Balding': 'Binary',
 'Mean Corpuscular Hemoglobin': 'Continous Measure',
 'Cholesterol': 'Continous Measure',
 'Prostate Cancer': 'Binary',
 'Epithelia Ovarian Cancer': 'Binary',
 'Primary Open Angle Glaucoma': 'Binary',
 "Alzheimer's Disease": 'Binary',
 'Tanning Ability': 'Binary',
 'Forced Expiratory Volume': 'Continous Measure',
 'Hip Circumference': 'Continous Measure',
 'Estimated Glomerular Filtration Rate': 'Continous Measure',
 'Depression': 'Binary',
 'Asthma Disease': 'Binary',
 'Hypothyroidism': 'Binary',
 'Venous Thromboembolic Disease': 'Binary',
 'Obesity': 'Binary',
 'Smoking Status': 'Binary',
 'Intelligence': 'Continous Measure',
 'Morning Person': 'Binary',
 'Dried Fruit Intake': 'Binary',
 'Rheumatoid Arthirits': 'Binary',
 'Systemic Lupus Erythematosus': 'Binary',
 'Osteoporosis': 'Binary',
 'Angina': 'Binary',
 'Red Blood Cell Distribution Width': 'Continous Measure',
 'Atrial Fibrillation': 'Binary',
 'Low-Density Lipoprotein': 'Continous Measure',
 'Mean Corpuscular Hemoglobin Concentration': 'Continous Measure',
 'Myxedema': 'Binary',
 'Age at Menarche': 'Continous Measure',
 'Supplementary Vitamin and Mineral': 'Binary',
 'Sing Back Musical Note': 'Continous Measure',
 'Sleep Duration': 'Continous Measure',
 'Bladder Cancer': 'Binary',
 'High-Density Lipoprotein': 'Continous Measure',
 'Hemoglobin A1c': 'Continous Measure'}

In [ ]:
#Makes a table which shows the number of times each method was comapred against each
#phenotype
traitsList = list(traits.keys())
traitCompDf = pd.DataFrame(columns=np.asarray(models['Method']))
for x in traitsList:
    tmparray = []
    for y in traits[x][0].columns:
        tmparray.append(sum(traits[x][0][y]))
    tmpArrayDf = pd.DataFrame([tmparray],columns=np.asarray(models['Method']))
    traitCompDf = pd.concat([traitCompDf,tmpArrayDf],ignore_index=True)


row_sums = traitCompDf.sum(axis=1)
rows_to_drop = traitCompDf[row_sums == 0].index
traitCompDf = traitCompDf.drop(rows_to_drop)
traitCompDf = traitCompDf.reset_index(drop=True)

modList = []
for x in range(len(row_sums)):
    if row_sums[x] != 0:
        modList.append(traitsList[x])
l = []
for x in modList:
    l.append(phenoTYPE[x])
traitCompDf.insert(loc=0, column='traits', value=modList)
traitCompDf.insert(loc=1, column='Data Type', value=l)

totalArray = []
totalArray.append("Total")
totalArray.append("Not Applicable")
for j in range(2,16):
   y = traitCompDf.columns[j]
   totalArray.append(sum(traitCompDf[y]))

totalArrayDf = pd.DataFrame([totalArray],columns=traitCompDf.columns)
traitCompDf = pd.concat([traitCompDf,totalArrayDf],ignore_index=True) 
traitCompDf = traitCompDf.sort_values(['Data Type',"traits"])
traitCompDf = traitCompDf.reset_index(drop=True)

traitCompDf.to_csv('traitCompAppliedDf.csv',index=False)

In [ ]:
#Creates a table showing the raw numbers on the head to head comparisons
comparison = np.zeros( (AA0.shape[1], AA0.shape[1]) , dtype=object)
for x in range(AA0.shape[1]):
    for y in range(AA0.shape[1]):
        if x!=y:
            tmp1 = np.asarray(AA0[AA0.columns[x]]) 
            tmp2 = np.asarray(AA0[AA0.columns[y]])
            tmp3 = np.asarray(WW0[WW0.columns[x]]) 
            tmp4 = np.multiply(tmp1,tmp2)
            comparison[x][y]= sum(np.multiply(tmp4,tmp3))

theCompDf = pd.DataFrame(comparison,columns=np.asarray(models['Method']))
for x in range(AA0.shape[1]):
    cVal=  sum(theCompDf.loc[x,])
    rVal = sum(theCompDf[theCompDf.columns[x]])
    comparison[x][x] = (cVal,rVal,cVal+rVal)
theCompDf = pd.DataFrame(comparison,columns=np.asarray(models['Method']))

theCompDf.to_csv('comparisonsApplied.csv',index=False)